In [3]:
from random import randint
import unittest
from os import system, name
from IPython.display import clear_output

# return the marks to be added and array after move
def moveArray(array):
    assert(len(array) == 4)
    result = []
    lastIsSumed = False
    marksToBeAdded = 0
    for num in array:
        if num != 0:
            if len(result) == 0:
                result.append(num) 
            elif result[-1] == num:
                if lastIsSumed:
                    result.append(num)
                    lastIsSumed = False
                else:
                    result[-1] += num
                    marksToBeAdded += 2 * num
                    lastIsSumed = True
            else:
                result.append(num)
                lastIsSumed = False
    result += [0] * (4 - len(result))
    return marksToBeAdded, result

# We define down and right as forward direction
def getIthArrayInd(i, isRow, isForward):
    margin = 1 if isRow else 4
    direction = -1 if isForward else 1
    return [i * (5 - margin) + j * margin for j in range(4)[::direction]]

class Obj_2048:
    
    directions = {'Right' : (True, True), 
                 'Down': (False, True),
                 'Left' : (True, False),
                 'Up': (False, False)}
    
    def reset(self):
        self._values = [0]*16
        self._mark = 0
        self._steps = 0
        
    def __init__(self):
        self.reset()
    
    def getMark(self):
        return self._mark
    
    def getSteps(self):
        return self._steps
    
    def getHighestNum(self):
        return max(self._values)
    
    def _unpackArray(self, indList, array):
        assert(len(indList) == 4)
        for i in range(4):
            self._values[indList[i]] = array[i]
         
    def _move(self,direction, isCheck):
        isRow, isForward = Obj_2048.directions[direction]
        marksToBeAdded = 0
        canBeMoved = False
        for i in range(4):
            indList = getIthArrayInd(i, isRow, isForward)
            array = [self._values[ind] for ind in indList]
            mark, result = moveArray(array)
            #means something changed
            if (array != result):
                canBeMoved = True                          
                #if it is a player's action, change the values
                if not isCheck:
                    print(array)
                    print(result)
                    print(indList)
                    print(str(i) + " above")
                    self._unpackArray(indList,result)
                marksToBeAdded += mark
        if (not isCheck) and canBeMoved:
            self._mark += marksToBeAdded
            self._steps += 1
        return canBeMoved
        
    def _isGameOver(self):
        for direction in Obj_2048.directions.keys():
            if (self._move(direction, True)):
                return False
        return True
    
    def _generateNext(self):
        tmp = []
        adj = [-4, -1 , 1, 4]
        for i in range(16):
            okToGenerate = False
            for j in adj:
                if i + j >= 16 or i + j < 0 or self._values[i] != 0:
                    okToGenerate = True
                    break
            if okToGenerate:
                tmp.append(i)
        self._values[tmp[randint(0,len(tmp) - 1)]] = 2 * (randint(0,1) + 1)
                    
    def play(self):
        ##get first 2 nums
        self._generateNext()
        self._generateNext()
        while(not self._isGameOver()):
            for i in range(4):
                print(self._values[i * 4: i * 4 + 4])
            print("marks: " + str(self._mark))
            print("moves: " + str(self._steps))
            direction = input("Enter next move:")
            clear_output(wait=True)
            if (direction == "exit"):
                break
            if (direction in Obj_2048.directions.keys()):
                if self._move(direction, False):
                    self._generateNext()
            else:
                print('\x1b[6;30;42m' + "Please input one of \"Right, Up, Down, Left\"" + '\x1b[0m', end = "\r")
        print("Game Over")
        print("marks: " + str(self._mark))
        print("moves: " + str(self._steps))
        print("highest: " + str(self.getHighestNum()))        

In [ ]:
game = Obj_2048()
game.play()

[0, 0, 0, 2]
[2, 0, 0, 0]
[1, 5, 9, 13]
1 above
[4, 2, 0, 8]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
marks: 16
moves: 4


In [1]:
import tensorflow as tf
import numpy as np
from copy import deepcopy
import random 
import math
import matplotlib.pyplot as plt

In [8]:
#first convolution layer depth
depth1 = 128

#second convolution layer depth
depth2 = 128

#batch size for batch gradient descent
batch_size = 512

#input units
input_units = 16

#fully connected layer neurons
hidden_units = 256

#output neurons = number of moves
output_units = 4


#hyper parameters
start_learning_rate = 0.0005

#gamma for Q-learning
gamma = 0.9

#epsilon greedy approach
epsilon = 0.9

#to store states and lables of the game for training
#states of the game
replay_memory = list()

#labels of the states
replay_labels = list()

#capacity of memory
mem_capacity = 6000

#deprecated method 
#todo: fix placeholder
import warnings
warnings.filterwarnings('ignore')
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

#input data
tf_batch_dataset = tf.placeholder(tf.float32,shape=(batch_size,4,4,16))
tf_batch_labels  = tf.placeholder(tf.float32,shape=(batch_size,output_units))

single_dataset   = tf.placeholder(tf.float32,shape=(1,4,4,16))

#CONV LAYERS
#conv layer1 weights
conv1_layer1_weights = tf.Variable(tf.truncated_normal([1,2,input_units,depth1],mean=0,stddev=0.01))
conv2_layer1_weights = tf.Variable(tf.truncated_normal([2,1,input_units,depth1],mean=0,stddev=0.01))

#conv layer2 weights
conv1_layer2_weights = tf.Variable(tf.truncated_normal([1,2,depth1,depth2],mean=0,stddev=0.01))
conv2_layer2_weights = tf.Variable(tf.truncated_normal([2,1,depth1,depth2],mean=0,stddev=0.01))

#FUllY CONNECTED LAYERS
expand_size = 2*4*depth2*2 + 3*3*depth2*2 + 4*3*depth1*2
fc_layer1_weights = tf.Variable(tf.truncated_normal([expand_size,hidden_units],mean=0,stddev=0.01))
fc_layer1_biases = tf.Variable(tf.truncated_normal([1,hidden_units],mean=0,stddev=0.01))
fc_layer2_weights = tf.Variable(tf.truncated_normal([hidden_units,output_units],mean=0,stddev=0.01))
fc_layer2_biases = tf.Variable(tf.truncated_normal([1,output_units],mean=0,stddev=0.01))

#model
def model(dataset):
    #layer1
    conv1 = tf.nn.conv2d(dataset,conv1_layer1_weights,[1,1,1,1],padding='VALID') 
    conv2 = tf.nn.conv2d(dataset,conv2_layer1_weights,[1,1,1,1],padding='VALID') 
    
    #layer1 relu activation
    relu1 = tf.nn.relu(conv1)
    relu2 = tf.nn.relu(conv2)
    
    #layer2
    conv11 = tf.nn.conv2d(relu1,conv1_layer2_weights,[1,1,1,1],padding='VALID') 
    conv12 = tf.nn.conv2d(relu1,conv2_layer2_weights,[1,1,1,1],padding='VALID') 

    conv21 = tf.nn.conv2d(relu2,conv1_layer2_weights,[1,1,1,1],padding='VALID') 
    conv22 = tf.nn.conv2d(relu2,conv2_layer2_weights,[1,1,1,1],padding='VALID') 

    #layer2 relu activation
    relu11 = tf.nn.relu(conv11)
    relu12 = tf.nn.relu(conv12)
    relu21 = tf.nn.relu(conv21)
    relu22 = tf.nn.relu(conv22)
    
    #get shapes of all activations
    shape1 = relu1.get_shape().as_list()
    shape2 = relu2.get_shape().as_list()
    
    shape11 = relu11.get_shape().as_list()
    shape12 = relu12.get_shape().as_list()
    shape21 = relu21.get_shape().as_list()
    shape22 = relu22.get_shape().as_list()

    #expansion
    hidden1 = tf.reshape(relu1,[shape1[0],shape1[1]*shape1[2]*shape1[3]])
    hidden2 = tf.reshape(relu2,[shape2[0],shape2[1]*shape2[2]*shape2[3]])
    
    hidden11 = tf.reshape(relu11,[shape11[0],shape11[1]*shape11[2]*shape11[3]])
    hidden12 = tf.reshape(relu12,[shape12[0],shape12[1]*shape12[2]*shape12[3]])
    hidden21 = tf.reshape(relu21,[shape21[0],shape21[1]*shape21[2]*shape21[3]])
    hidden22 = tf.reshape(relu22,[shape22[0],shape22[1]*shape22[2]*shape22[3]])

    #concatenation
    hidden = tf.concat([hidden1,hidden2,hidden11,hidden12,hidden21,hidden22],axis=1)

    #full connected layers
    hidden = tf.matmul(hidden,fc_layer1_weights) + fc_layer1_biases
    hidden = tf.nn.relu(hidden)

    #output layer
    output = tf.matmul(hidden,fc_layer2_weights) + fc_layer2_biases
    
    #return output
    return output

#for single example
single_output = model(single_dataset)

#for batch data
logits = model(tf_batch_dataset)

#loss
loss = tf.square(tf.subtract(tf_batch_labels,logits))
loss = tf.reduce_sum(loss,axis=1,keep_dims=True)
loss = tf.reduce_mean(loss)/2.0

#optimizer
global_step = tf.Variable(0)  # count the number of steps taken.
learning_rate = tf.train.exponential_decay(float(start_learning_rate), global_step, 1000, 0.90, staircase=True)
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss, global_step=global_step)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
#loss
J = []

#scores
scores = []

#to store final parameters
final_parameters = {}

#number of episodes
M = 200001